**GRADIO UI AND FASTAPI + FRONTEND**

1.   Mounting of Drive to Colab




In [1]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [2]:
# Mount Google Drive (interactive prompt)
drive.mount('/content/drive')

NameError: name 'drive' is not defined

Checks if Drive mounting in Colab is succesful.

In [ ]:
import os

drive_path = '/content/drive/My Drive'

if os.path.exists(drive_path):
    print(f"Listing contents of '{drive_path}':")

else:
    print(f"Google Drive is not mounted at '{drive_path}'. Please ensure it is mounted correctly.")

Listing contents of '/content/drive/My Drive':


**Path Definition and connection**

Defines paths for model directory, (fastAPI) app1 file, and requirements file in Google Drive.

Copies these files/folder into the Colab working directory.

Prints status messages for success or warnings.

Lists the files in /content/ to confirm the copy.

In [ ]:
import os

drive_base_path = '/content/drive/My Drive'
current_working_directory = '/content/'

# Define source paths for the files/directory to be copied
source_model_dir = os.path.join(drive_base_path, 'distilbert_ticket_classifier_model')
source_app_file = os.path.join(drive_base_path, 'app1.py')
source_requirements_file = os.path.join(drive_base_path, 'requirements.txt')

dest_model_dir = os.path.join(current_working_directory, 'distilbert_ticket_classifier_model')
dest_app_file = os.path.join(current_working_directory, 'app1.py')
dest_requirements_file = os.path.join(current_working_directory, 'requirements.txt')

if os.path.exists(source_model_dir):
    !cp -r "{source_model_dir}" "{current_working_directory}"
    print(f"Copied directory: {source_model_dir} to {current_working_directory}")
else:
    print(f"Warning: Model directory not found at {source_model_dir}")

if os.path.exists(source_app_file):
    !cp "{source_app_file}" "{current_working_directory}"
    print(f"Copied file: {source_app_file} to {current_working_directory}")
else:
    print(f"Warning: app1.py not found at {source_app_file}")

if os.path.exists(source_requirements_file):
    !cp "{source_requirements_file}" "{current_working_directory}"
    print(f"Copied file: {source_requirements_file} to {current_working_directory}")
else:
    print(f"Warning: requirements.txt not found at {source_requirements_file}")

print("\nVerifying copied files in current directory:")
!ls -F {current_working_directory}

Copied directory: /content/drive/My Drive/distilbert_ticket_classifier_model to /content/
Copied file: /content/drive/My Drive/app1.py to /content/
Copied file: /content/drive/My Drive/requirements.txt to /content/

Verifying copied files in current directory:
app1.py				     drive/	       sample_data/
distilbert_ticket_classifier_model/  requirements.txt


Installing requirements dependencies

In [ ]:
print("Installing dependencies from requirements.txt...")
!pip install -r requirements.txt
print("Dependencies installed.")

Installing dependencies from requirements.txt...
Dependencies installed.


In [ ]:
!pip install fastapi uvicorn pyngrok transformers torch

In [ ]:
!ngrok config add-authtoken 35J2rLltYUXssviyY8GO8LQwTwI_3x4iP69zCTnjfKh1ivEkx

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


**The Gradio Interface**

The code allows users to enter a customer support ticket and immediately see all possible categories with their confidence levels, helping support teams quickly route or respond to tickets efficiently

In [ ]:
import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load Model and Tokenizer
model_path = "/content/drive/MyDrive/distilbert_ticket_classifier_model"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)
model.eval()

#  Label Mapping
label_map = {
    0: "account",
    1: "billing",
    2: "other",
    3: "technical"
}

# Prediction Function with Highlighted Top Category
def classify_ticket_text(text):
    if not text.strip():
        return "Error: Please enter valid text."

    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)

        if hasattr(outputs, 'logits'):
            logits = outputs.logits
            probs = torch.nn.functional.softmax(logits, dim=1)
        else:
            probs = outputs[0]

        # Convert to percentages and clamp
        probs_percentage = [round(min(max(p.item(), 0), 1) * 100, 2) for p in probs[0]]

    # Pair labels with probabilities
    label_probs = [(label_map[i], probs_percentage[i]) for i in range(len(probs_percentage))]
    # Sort by confidence descending
    label_probs.sort(key=lambda x: x[1], reverse=True)

    # Build Markdown string with top category highlighted
    result_lines = []
    for i, (label, prob) in enumerate(label_probs):
        if i == 0:
            # Top predicted category
            result_lines.append(f"✅ **{label}: {prob}%**")
        else:
            result_lines.append(f"- {label}: {prob}%")

    result_markdown = "\n".join(result_lines)
    return result_markdown

# Gradio Interface
demo = gr.Interface(
    fn=classify_ticket_text,
    inputs=gr.Textbox(lines=3, placeholder="Enter customer ticket text...", label="Ticket Text"),
    outputs=gr.Markdown(label="Category Confidence (%)"),
    title=" Customer Support Ticket Classifier",
    description="Enter a customer support ticket and see all possible categories with their confidence levels.",
    examples=[
        ["I can't log into my account"],
        ["My payment failed but I was charged"],
        ["The app keeps crashing after update"],
        ["Just wanted to say thanks for the great support!"]
    ],
    theme="default"
)

demo.launch(debug=True)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://1d795ad06198028eae.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://1d795ad06198028eae.gradio.live


**Deployment**

This code deploys your FastAPI app temporarily online using ngrok, so external systems (like your HTML page or teammates) can send requests to it — even when running locally or in Google Colab.

In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn
from app1 import app

# Allow nested async loops (needed for Colab)
nest_asyncio.apply()

# Create ngrok tunnel
public_url = ngrok.connect(8000).public_url
print("Public API URL:", public_url)

# uvicorn server configuration
config = uvicorn.Config(app=app, host="0.0.0.0", port=8000)
server = uvicorn.Server(config)

# Run server (non-blocking)
await server.serve()


Public API URL: https://unblossoming-unencouraging-cash.ngrok-free.dev


INFO:     Started server process [291]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     154.161.240.229:0 - "GET / HTTP/1.1" 200 OK
INFO:     154.161.240.229:0 - "OPTIONS /predict HTTP/1.1" 200 OK
INFO:     154.161.240.229:0 - "POST /predict HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [291]
